In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
#sys function provides the name of the existing python modules which have been imported.
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')



In [ ]:
import os
#The OS module in Python provides functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory, etc.
from pathlib import Path
#Pathlib module in Python provides various classes representing file system paths with semantics appropriate for different operating systems. 
import glob
#the glob module is used to retrieve files/pathnames matching a specified pattern.
from tqdm import tqdm
#tqdm is used to display bar library with good support for nested loops and Jupyter/IPython notebooks.
tqdm.pandas()
#when tqdm.pandas() is used then you can simply replace all your .apply() functions with .progress_apply() it’s really that simple!
import json
#for reading json file and for mapping operation.
import pandas as pd
#for reading the reading the data of csv file and converting into pandas dataframe
import numpy as np
#numpy array used for scientific computational purpose.

import imagehash
#using different algorithm of imagehash to detect two particular images are similar or not.


import seaborn as sns
import matplotlib.pyplot as plt
#using seaborn and matplotlib for visulization and different plotting purpose.
import cv2
#Cv2 is computervison library for reading the images.
from PIL import Image
#PIL is python imaging library for reading the images.


import albumentations
#albumentations for augmentation purpose
from albumentations.pytorch.transforms import ToTensorV2
#ToTensorV2 convert image and mask to torch.Tensor

import torch
import timm
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

from keras.preprocessing.image import load_img
#load the image in PIL format.
from keras.preprocessing.image import img_to_array 
#Converts a PIL Image instance to a Numpy array.
from keras.applications.resnet50 import preprocess_input 
#Preprocesses a tensor or Numpy array encoding a batch of images.(RGB---->BGR && floating point--->>float32)


from keras.applications.resnet50 import ResNet50
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score










In [ ]:
base_dir=Path("../input/cassava-leaf-disease-classification")
Base_dir="../input/cassava-leaf-disease-classification"

In [ ]:
df=pd.read_csv(base_dir/"train.csv")
df

In [ ]:
with open(base_dir/"label_num_to_disease_map.json") as f:
    mapping=json.loads(f.read())
    mapping={int(k):v for k,v in mapping.items()}
mapping

In [ ]:
df["diseases_label"]=df["label"].map(mapping)
df

In [ ]:
def plot_images(label,diseases_label,count,mode=0):
    plot_list = df[df["label"] == label].sample(count)['image_id'].tolist()
    
    # Printing list of images
    if mode:
        print(plot_list)
        
    labels = [diseases_label for i in range(len(plot_list))]
    size = np.sqrt(count)
    if int(size)*int(size) < count:
        size = int(size) + 1
        
    plt.figure(figsize=(20, 20))
    
    for idx, (image_id, label) in enumerate(zip(plot_list, labels)):
        plt.subplot(size, size, idx + 1)
        image = cv2.imread(str(base_dir/"train_images"/image_id))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        plt.imshow(image)
        plt.title(label, fontsize=12)
        plt.axis("off")
    
    plt.show()
    
    

In [ ]:
plot_images(0,"Cassava Bacterial Blight (CBB)",9,1)

In [ ]:
plot_images(1,"Cassava Brown Streak Disease (CBSD)",9,1)

In [ ]:
plot_images(2,"Cassava Green Mottle (CGM)",9,1)

In [ ]:
plot_images(3,"Cassava Mosaic Disease (CMD)",9,1)

In [ ]:
print("NUMBER OF CMD IMAGES IN THE TRAINING DATASET:")
df[df["label"]==3]["image_id"].count()

In [ ]:
plot_images(4,"Healthy",9,1)

In [ ]:
print("NUMBER OF HEAlTHY IMAGES IN THE TRAINING DATASET:")
df[df["label"]==4]["image_id"].count()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(y='diseases_label',data = df)
plt.show()

In [ ]:
def extract_features(image_id, model):
    file = str(base_dir/'train_images'/image_id)
    # load the image as an instance of PIL
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    
    return features



In [ ]:
base_model = ResNet50()
# create the base pre-trained model

model = Model(inputs = base_model.inputs, outputs = base_model.layers[-2].output)
# this is the model we will train


healthy = df[df['label']==4]
healthy['features'] = healthy['image_id'].progress_apply(lambda x:extract_features(x,model))



In [ ]:
np.array(healthy['features'].values.tolist()).reshape(-1,2048)

In [ ]:
np.array(healthy["image_id"].values.tolist())

In [ ]:
features=np.array(healthy['features'].values.tolist()).reshape(-1,2048)
image_ids=np.array(healthy["image_id"].values.tolist())

In [ ]:
kmeans = KMeans(n_clusters=5,n_jobs=-1, random_state=22)
kmeans.fit(features)
kmeans.labels_



In [ ]:
group1 = {}

for file, cluster in zip(image_ids,kmeans.labels_):
    if cluster not in group1.keys():
        group1[cluster] = []
        group1[cluster].append(file)
    else:
        group1[cluster].append(file)


In [ ]:
group1


In [ ]:
def view_cluster(cluster):
    plt.figure(figsize=(25,25))
    files=group1[cluster]
    if len(files)>25:
        print(f"The number of sample images is reduced from {len(files)} to 25")
        rand=np.random.randint(0,len(files))
        files=files[rand:rand+25]
        for index,file in enumerate(files):
            plt.subplot(5,5,index+1)
            image=cv2.imread(Base_dir+"/train_images/"+file)
            image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            plt.imshow(image)
            plt.axis("off")
            plt.title(file)
        plt.show()


In [ ]:
view_cluster(0)

In [ ]:
view_cluster(1)

In [ ]:
view_cluster(2)

In [ ]:
view_cluster(3)

In [ ]:
view_cluster(4)

In [ ]:
cbsd= df[df['label']==1]
cbsd['features'] = cbsd['image_id'].progress_apply(lambda x:extract_features(x,model))

In [ ]:
cbsd_features=np.array(cbsd['features'].values.tolist()).reshape(-1,2048)
cbsd_image_ids=np.array(cbsd["image_id"].values.tolist())

In [ ]:
cbsd_kmeans = KMeans(n_clusters=5,n_jobs=-1, random_state=22)
cbsd_kmeans.fit(cbsd_features)
cbsd_kmeans.labels_

In [ ]:
group2 = {}

for file, cluster in zip(cbsd_image_ids,cbsd_kmeans.labels_):
    if cluster not in group2.keys():
        group2[cluster] = []
        group2[cluster].append(file)
    else:
        group2[cluster].append(file)

In [ ]:
group2

In [ ]:
def view_cluster(cluster):
    plt.figure(figsize=(25,25))
    files=group2[cluster]
    if len(files)>25:
        print(f"The number of sample images is reduced from {len(files)} to 25")
        rand=np.random.randint(0,len(files))
        files=files[rand:rand+25]
        for index,file in enumerate(files):
            plt.subplot(5,5,index+1)
            image=cv2.imread(Base_dir+"/train_images/"+file)
            image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            plt.imshow(image)
            plt.axis("off")
            plt.title(file)
        plt.show()

In [ ]:
view_cluster(0)

In [ ]:
view_cluster(1)

In [ ]:
view_cluster(2)

In [ ]:
view_cluster(3)

In [ ]:
view_cluster(4)

In [ ]:
train_image_path=[os.path.join(Base_dir+"/train_images/",k)for k in df.image_id.values]
train_targets=df.label.values

print(len(train_image_path))
print(len(train_targets))
train_image_path


In [ ]:
from torchvision.transforms import transforms
class CassavaDataset(Dataset):
    def __init__(self,image_ids,labels,dimension=None,augmentations=None):
        super().__init__()
        self.image_ids=image_ids
        self.labels=labels
        self.dim=dimension
        self.augmentations=augmentations
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self,idx):
        imge=cv2.imread(self.image_ids[idx])
        imge=cv2.cvtColor(imge,cv2.COLOR_BGR2RGB)
        
        if self.dim:
            imge=cv2.resize(imge,self.dim)
            
        if self.augmentations:
            aug_img=self.augmentations(image=imge)
            imge=aug_img["image"]
            
        return {"image":transforms.ToTensor()(imge),
                "label":torch.tensor(self.labels[idx])}    


In [ ]:
##---------